# Over- and Under-sampling Techniques (SMOTE, ADASYN, GNI, TimeGAN, Tomek Links, Random Under Sampling)

In [1]:
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import TomekLinks
import numpy as np

## SMOTE and ADASYN (with RUS and TomekLinks)

In [2]:
# SMOTE, TomekLinks, and RUS
# ADASYN, TomekLinks, and RUS

def rus_tomek_smote_adasyn(start_position, end_position, data_dir, name, X_train, Y_train):

    for i in range(start_position-1, end_position):

        num_fl_to_oversample = 10000
        num_nf_to_keep = 10000
        
        class_0_indices = np.where(Y_train[i] == 0)[0]
        class_1_indices = np.where(Y_train[i] == 1)[0]
        selected_class_0_indices = np.random.choice(class_0_indices, size=num_nf_to_keep, replace=False)
        new_indices = np.concatenate((selected_class_0_indices, class_1_indices))
        X_train_with_RUS = X_train[i][new_indices]
        Y_train_with_RUS = Y_train[i][new_indices]

        tomek_links = TomekLinks(sampling_strategy='auto')
        X_train_RUS_Tomek, Y_train_RUS_Tomek = tomek_links.fit_resample(X_train_with_RUS, Y_train_with_RUS)


        smote = SMOTE(sampling_strategy='minority', random_state=42)
        X_train_RUS_Tomek_Smote, Y_train_RUS_Tomek_Smote = smote.fit_resample(X_train_RUS_Tomek, Y_train_RUS_Tomek)

        adasyn = ADASYN(sampling_strategy='minority', random_state=42)
        X_train_RUS_Tomek_Adasyn, Y_train_RUS_Tomek_Adasyn = adasyn.fit_resample(X_train_RUS_Tomek, Y_train_RUS_Tomek)

        print('Partition' + str(i+1) + ': ')
        print(X_train_RUS_Tomek_Smote.shape)
        print(Y_train_RUS_Tomek_Smote.shape)
        print(X_train_RUS_Tomek_Adasyn.shape)
        print(str(Y_train_RUS_Tomek_Adasyn.shape) + '\n')

        
        num_samples = X_train_RUS_Tomek_Smote.shape[0]
        shuffle_indices = np.random.permutation(num_samples)

        X_train_RUS_Tomek_Smote_shuffled = X_train_RUS_Tomek_Smote[shuffle_indices]
        Y_train_RUS_Tomek_Smote_shuffled = Y_train_RUS_Tomek_Smote[shuffle_indices]
    
        with open(data_dir + "RUS_Tomek_Smote" + "_Partition" + str(i+1) 
                       + "_" + name + ".pkl", 'wb') as f:
            pickle.dump(X_train_RUS_Tomek_Smote_shuffled, f)

        with open(data_dir + "RUS_Tomek_Smote" + "_Partition" + str(i+1) 
                       + "_Labels_" + name + ".pkl", 'wb') as f:
            pickle.dump(Y_train_RUS_Tomek_Smote_shuffled, f)
            
        
        num_samples = X_train_RUS_Tomek_Adasyn.shape[0]
        shuffle_indices = np.random.permutation(num_samples)

        X_train_RUS_Tomek_Adasyn_shuffled = X_train_RUS_Tomek_Adasyn[shuffle_indices]
        Y_train_RUS_Tomek_Adasyn_shuffled = Y_train_RUS_Tomek_Adasyn[shuffle_indices]
    
        with open(data_dir + "RUS_Tomek_Adasyn" + "_Partition" + str(i+1) 
                       + "_" + name + ".pkl", 'wb') as f:
            pickle.dump(X_train_RUS_Tomek_Adasyn_shuffled, f)

        with open(data_dir + "RUS_Tomek_Adasyn" + "_Partition" + str(i+1) 
                       + "_Labels_" + name + ".pkl", 'wb') as f:
            pickle.dump(Y_train_RUS_Tomek_Adasyn_shuffled, f)



## GNI (with RUS and TomekLinks)

In [3]:
import numpy as np
import pickle
from imblearn.under_sampling import TomekLinks
from imblearn.over_sampling import SMOTE

def rus_tomek_gni(start_position, end_position, data_dir, name, X_train, Y_train, noise_proportion=0.05):
    for i in range(start_position - 1, end_position):
        # Random Under Sampling
        num_nf_to_keep = 10000
        class_0_indices = np.where(Y_train[i] == 0)[0]
        class_1_indices = np.where(Y_train[i] == 1)[0]
        selected_class_0_indices = np.random.choice(class_0_indices, size=num_nf_to_keep, replace=False)
        new_indices = np.concatenate((selected_class_0_indices, class_1_indices))
        X_train_RUS = X_train[i][new_indices]
        Y_train_RUS = Y_train[i][new_indices]

        # Tomek Links
        tomek_links = TomekLinks(sampling_strategy='auto')
        X_train_RUS_Tomek, Y_train_RUS_Tomek = tomek_links.fit_resample(X_train_RUS, Y_train_RUS)

        # Gaussian Noise Injection
        std_dev = np.std(X_train_RUS_Tomek, axis=0)
        noise_level = std_dev * noise_proportion
        class_counts = np.bincount(Y_train_RUS_Tomek.astype(int))
        max_class_count = np.max(class_counts)
        minority_class = np.argmin(class_counts)
        num_samples_to_add = max_class_count - class_counts[minority_class]
        minority_indices = np.where(Y_train_RUS_Tomek == minority_class)[0]

        new_samples = []
        for _ in range(num_samples_to_add):
            sample_index = np.random.choice(minority_indices)
            sample = X_train_RUS_Tomek[sample_index]
            noise = np.random.normal(0, noise_level, sample.shape)
            new_sample = sample + noise
            new_samples.append(new_sample)

        X_train_augmented = np.vstack((X_train_RUS_Tomek, new_samples))
        Y_train_augmented = np.append(Y_train_RUS_Tomek, np.full(num_samples_to_add, minority_class))

        print('Partition' + str(i+1) + ': ')
        print(X_train_augmented.shape)
        print(str(Y_train_augmented.shape) + '\n')
        
        # Shuffle and save the augmented data
        shuffle_indices = np.random.permutation(len(Y_train_augmented))
        X_train_augmented = X_train_augmented[shuffle_indices]
        Y_train_augmented = Y_train_augmented[shuffle_indices]

        with open(data_dir + "RUS_Tomek_GNI" + "_Partition" + str(i+1) + "_" + name + ".pkl", 'wb') as f:
            pickle.dump(X_train_augmented, f)

        with open(data_dir + "RUS_Tomek_GNI" + "_Partition" + str(i+1) + "_Labels_" + name + ".pkl", 'wb') as f:
            pickle.dump(Y_train_augmented, f)


## TimeGAN (with RUS and TomekLinks)

In [4]:
import numpy as np
import pickle
from imblearn.under_sampling import TomekLinks
from timegan import timegan

def rus_tomek_timegan(start_position, end_position, data_dir, name, X_train, Y_train):
    # TimeGAN network parameters
    timegan_params = dict()
    timegan_params['module'] = 'gru'
    timegan_params['hidden_dim'] = 24
    timegan_params['num_layer'] = 3
    timegan_params['iterations'] = 8000
    timegan_params['batch_size'] = 128

    for i in range(start_position-1, end_position):
        # Parameters for Random Under Sampling
        num_nf_to_keep = 10000

        # Random Under Sampling
        class_0_indices = np.where(Y_train[i] == 0)[0]
        class_1_indices = np.where(Y_train[i] == 1)[0]
        selected_class_0_indices = np.random.choice(class_0_indices, size=num_nf_to_keep, replace=False)
        new_indices = np.concatenate((selected_class_0_indices, class_1_indices))
        X_train_with_RUS = X_train[i][new_indices]
        Y_train_with_RUS = Y_train[i][new_indices]

        # Apply Tomek Links
        tomek_links = TomekLinks(sampling_strategy='auto')
        X_train_RUS_Tomek, Y_train_RUS_Tomek = tomek_links.fit_resample(X_train_with_RUS, Y_train_with_RUS)
        
        num_attributes = 24
        num_partitions = 5
        num_timestamps = 60
        X_train_RUS_Tomek_3D = []
        new_3D = np.zeros((X_train_RUS_Tomek.shape[0], num_timestamps, num_attributes))

        for j in range(0, X_train_RUS_Tomek.shape[0]):
            for m in range(0, num_attributes):
                new_3D[j,:,m] = X_train_RUS_Tomek[j,m*num_timestamps:(m+1)*num_timestamps]
        X_train_RUS_Tomek_3D = new_3D
            
        del X_train_RUS_Tomek

        # Identify the minority class after RUS and Tomek Links
        class_counts = np.bincount(Y_train_RUS_Tomek.astype(int))
        minority_class = np.argmin(class_counts)
        minority_indices = np.where(Y_train_RUS_Tomek == minority_class)[0]
        minority_class_data = X_train_RUS_Tomek_3D[minority_indices]

        # Number of data to be generated by TimeGAN
        num_of_data_to_be_generated = len(Y_train_RUS_Tomek) - 2 * len(minority_indices)

        # Generate synthetic data using TimeGAN
        generated_data = timegan(minority_class_data, timegan_params, num_of_data_to_be_generated)

        # Combine the original data with generated data
        X_train_augmented = np.vstack((X_train_RUS_Tomek_3D, generated_data))
        Y_train_augmented = np.append(Y_train_RUS_Tomek, np.full(len(generated_data), minority_class))
        
        num_timestamps = 60
        num_attributes = 24
        new_partition = np.zeros((np.array(X_train_augmented).shape[0], num_timestamps*(num_attributes)))        
        
        for l in range(0,new_partition.shape[0]):
            new_column = np.zeros((num_timestamps,num_attributes)) 
            new_column = X_train_augmented[l,:,:]

            flettened = np.zeros(num_timestamps*(num_attributes))

            for n in range(0,num_attributes):
                flettened[(n)*num_timestamps:(n+1)*num_timestamps] = new_column[:,n]

            new_partition[l,:] = flettened

        print('Partition' + str(i+1) + ': ')
        print(new_partition.shape)
        print(str(Y_train_augmented.shape) + '\n')
        
        # Shuffle the augmented dataset
        shuffle_indices = np.random.permutation(len(Y_train_augmented))
        X_augmented_shuffled = new_partition[shuffle_indices]
        Y_augmented_shuffled = Y_train_augmented[shuffle_indices]

        # Save the augmented data
        with open(data_dir + "RUS_Tomek_TimeGAN" + "_Partition" + str(i+1) + "_" + name + ".pkl", 'wb') as f:
            pickle.dump(X_augmented_shuffled, f)
        with open(data_dir + "RUS_Tomek_TimeGAN" + "_Partition" + str(i+1) + "_Labels_" + name + ".pkl", 'wb') as f:
            pickle.dump(Y_augmented_shuffled, f)


### Reading the Data built by the Previous Notebooks (FPCKNN Imputation and Without B and C Classes)

In [5]:
import pickle
import pandas as pd

data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/4_3_FinalData_WithoutB&C_Concatenation_KnnImputation/"

X_train_array = []
Y_train_array = []

num_partitions = 5
for i in range(0, num_partitions):
    with open(data_dir + "Partition" + str(i+1) + "_WithoutB&C_Concatenation_KnnImputation" +".pkl", 'rb') as f:
        X_train_array.append(pickle.load(f))
    with open(data_dir + "Partition" + str(i+1) + "_Labels_WithoutB&C_Concatenation_KnnImputation" +".pkl", 'rb') as f:
        Y_train_array.append(pickle.load(f))

## Running the Sampling Techniques

In [7]:
data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/\
6_1_OUSampling_Concatenation_KnnImputation/"

name = "OUSampling_WithoutB&C_Concatenation_KnnImputation"

rus_tomek_smote_adasyn(1, 5, data_dir, name, X_train_array, Y_train_array)

Partition1: 
(19996, 1440)
(19996,)
(19920, 1440)
(19920,)

Partition2: 
(19994, 1440)
(19994,)
(19922, 1440)
(19922,)

Partition3: 
(19994, 1440)
(19994,)
(19965, 1440)
(19965,)

Partition4: 
(19998, 1440)
(19998,)
(20013, 1440)
(20013,)

Partition5: 
(19998, 1440)
(19998,)
(19954, 1440)
(19954,)



In [8]:
data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/\
6_1_OUSampling_Concatenation_KnnImputation/"

name = "OUSampling_WithoutB&C_Concatenation_KnnImputation"

rus_tomek_gni(1, 5, data_dir, name, X_train_array, Y_train_array)

Partition1: 
(20000, 1440)
(20000,)

Partition2: 
(19998, 1440)
(19998,)

Partition3: 
(19992, 1440)
(19992,)

Partition4: 
(20000, 1440)
(20000,)

Partition5: 
(19996, 1440)
(19996,)



In [6]:
data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/\
6_1_OUSampling_Concatenation_KnnImputation/"

name = "OUSampling_WithoutB&C_Concatenation_KnnImputation"

rus_tomek_timegan(1, 5, data_dir, name, X_train_array, Y_train_array)

Start Embedding Network Training
step: 0/8000, e_loss: 0.2488
step: 1000/8000, e_loss: 0.0535
step: 2000/8000, e_loss: 0.036
step: 3000/8000, e_loss: 0.031
step: 4000/8000, e_loss: 0.0257
step: 5000/8000, e_loss: 0.0238
step: 6000/8000, e_loss: 0.0237
step: 7000/8000, e_loss: 0.0205
Finish Embedding Network Training
Start Training with Supervised Loss Only
step: 0/8000, s_loss: 0.2179
step: 1000/8000, s_loss: 0.023
step: 2000/8000, s_loss: 0.0152
step: 3000/8000, s_loss: 0.0121
step: 4000/8000, s_loss: 0.0111
step: 5000/8000, s_loss: 0.0109
step: 6000/8000, s_loss: 0.0101
step: 7000/8000, s_loss: 0.0079
Finish Training with Supervised Loss Only
Start Joint Training
step: 0/8000, d_loss: 2.3817, g_loss_u: 0.4814, g_loss_s: 0.0171, g_loss_v: 0.3251, e_loss_t0: 0.0466
step: 1000/8000, d_loss: 1.2672, g_loss_u: 2.027, g_loss_s: 0.0112, g_loss_v: 0.0395, e_loss_t0: 0.0179
step: 2000/8000, d_loss: 0.8036, g_loss_u: 3.296, g_loss_s: 0.0199, g_loss_v: 0.0526, e_loss_t0: 0.0168
step: 3000/8000,